# Import Required Libraries
Import the necessary libraries, including requests and pandas.

In [39]:
# Import Required Libraries
import requests
import pandas as pd

# Additional libraries for HTML rendering
from IPython.display import HTML

# Download Premier League Game Results
Use the requests library to download the Premier League game results from football-data.org.

In [40]:
# Download Premier League Game Results
with open('api_token.txt', 'r') as file:
    api_token = file.read().strip()
url = 'https://api.football-data.org/v4/competitions/PL/matches'

headers = {
    'X-Auth-Token': api_token
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    matches = data['matches']
    # Convert the list of matches to a DataFrame
    df_view = pd.DataFrame(matches)
else:
    print(f"Failed to retrieve data: {response.status_code}")

# Parse JSON Data
Parse the JSON data received from the API and convert it into a pandas DataFrame.

In [41]:
df_matches = pd.DataFrame(response.json()['matches'])

In [42]:
# Parse JSON Data
# Extract relevant columns from the DataFrame
df_view = pd.DataFrame(df_matches)
df_view = df_view[['utcDate', 'homeTeam', 'awayTeam', 'score']]

# Normalize nested JSON columns
df_view['homeTeam'] = df_view['homeTeam'].apply(lambda x: x['tla'])
df_view['awayTeam'] = df_view['awayTeam'].apply(lambda x: x['tla'])
df_view['score'] = df_view['score'].apply(
    lambda x: f"{x['fullTime']['home']} - {x['fullTime']['away']}" 
    if x['fullTime']['home'] is not None and x['fullTime']['away'] is not None 
    else ""
)

# Rename columns for clarity
df_view.columns = ['Date', 'Home Team', 'Away Team', 'Score']

# Convert 'Date' column to datetime
df_view['Date'] = pd.to_datetime(df_view['Date'])

# Display the DataFrame
df_view

,Date,Home Team,Away Team,Score
0,2024-08-16 19:00:00+00:00,MUN,FUL,1 - 0
1,2024-08-17 11:30:00+00:00,IPS,LIV,0 - 2
2,2024-08-17 14:00:00+00:00,ARS,WOL,2 - 0
3,2024-08-17 14:00:00+00:00,EVE,BHA,0 - 3
4,2024-08-17 14:00:00+00:00,NEW,SOU,1 - 0
...,...,...,...,...
375,2025-05-25 15:00:00+00:00,NEW,EVE,
376,2025-05-25 15:00:00+00:00,NOT,CHE,
377,2025-05-25 15:00:00+00:00,SOU,ARS,
378,2025-05-25 15:00:00+00:00,TOT,BHA,


In [43]:
# Extract team crests from df_matches
teams_data = []
for match in df_matches['homeTeam']:
    teams_data.append({
        'Team': match['tla'],
        'Crest': match['crest']
    })

# Convert the list of dictionaries to a DataFrame
df_crests = pd.DataFrame(teams_data).drop_duplicates().reset_index(drop=True)

# Display the DataFrame
df_crests

,Team,Crest
0,MUN,https://crests.football-data.org/66.png
1,IPS,https://crests.football-data.org/349.png
2,ARS,https://crests.football-data.org/57.png
3,EVE,https://crests.football-data.org/62.png
4,NEW,https://crests.football-data.org/67.png
5,NOT,https://crests.football-data.org/351.png
6,WHU,https://crests.football-data.org/563.png
7,BRE,https://crests.football-data.org/402.png
8,CHE,https://crests.football-data.org/61.png
9,LEI,https://crests.football-data.org/338.png


# Render List of Games to HTML
Use pandas to render the DataFrame as an HTML table and display it in the notebook.

In [44]:
# Define a function to determine the winning team and format the HTML
def highlight_winner(row):
    try:
        home_score, away_score = map(int, row['Score'].split(' - '))
        if home_score > away_score:
            return f"<span style='color: green;'>{row['Home Team']} 🟢</span> vs {row['Away Team']}"
        elif away_score > home_score:
            return f"{row['Home Team']} vs <span style='color: green;'>{row['Away Team']} 🟢</span>"
        else:
            return f"{row['Home Team']} vs {row['Away Team']}"
    except ValueError:
        return f"{row['Home Team']} vs {row['Away Team']}"

# Apply the function to each row
df_view['Match'] = df_view.apply(highlight_winner, axis=1)

# Select only the columns to display
df_view_display = df_view[['Date', 'Match', 'Score']]

# Render the DataFrame to HTML
html_output = df_view_display.to_html(escape=False, index=False)
HTML(html_output)

Date,Match,Score
2024-08-16 19:00:00+00:00,MUN 🟢 vs FUL,1 - 0
2024-08-17 11:30:00+00:00,IPS vs LIV 🟢,0 - 2
2024-08-17 14:00:00+00:00,ARS 🟢 vs WOL,2 - 0
2024-08-17 14:00:00+00:00,EVE vs BHA 🟢,0 - 3
2024-08-17 14:00:00+00:00,NEW 🟢 vs SOU,1 - 0
2024-08-17 14:00:00+00:00,NOT vs BOU,1 - 1
2024-08-17 16:30:00+00:00,WHU vs AVL 🟢,1 - 2
2024-08-18 13:00:00+00:00,BRE 🟢 vs CRY,2 - 1
2024-08-18 15:30:00+00:00,CHE vs MCI 🟢,0 - 2
2024-08-19 19:00:00+00:00,LEI vs TOT,1 - 1


In [45]:
# Specify the 3 teams
teams_of_interest = ['ARS', 'CHE', 'LIV', 'NOT']

# Extract all unique teams from the matches
all_teams = set(df_view['Home Team']).union(set(df_view['Away Team']))

# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame(index=list(all_teams), columns=pd.MultiIndex.from_product([teams_of_interest, ['🏠Home', '✈️Away']], names=['Team', 'Location']))

# Populate the DataFrame with scores
for team in all_teams:
    for opponent in teams_of_interest:
        if team != opponent:
            # Home game
            home_game = df_view[(df_view['Home Team'] == team) & (df_view['Away Team'] == opponent)]
            if not home_game.empty:
                # Swap the score to show from the opponent's point of view
                score = home_game['Score'].values[0]
                if score:
                    home_score, away_score = score.split(' - ')
                    results_df.loc[team, (opponent, '🏠Home')] = f"{away_score} - {home_score}"
                else:
                    results_df.loc[team, (opponent, '🏠Home')] = '📆 ' + home_game['Date'].dt.strftime('%m/%d').values[0]
            else:
                results_df.loc[team, (opponent, '🏠Home')] = ''
            
            # Away game
            away_game = df_view[(df_view['Home Team'] == opponent) & (df_view['Away Team'] == team)]
            if not away_game.empty:
                if away_game['Score'].values[0]:
                    results_df.loc[team, (opponent, '✈️Away')] = away_game['Score'].values[0]
                else:
                    results_df.loc[team, (opponent, '✈️Away')] = '📆 ' + away_game['Date'].dt.strftime('%m/%d').values[0]
            else:
                results_df.loc[team, (opponent, '✈️Away')] = ''
        else:
            results_df.loc[team, (opponent, '🏠Home')] = 'N/A'
            results_df.loc[team, (opponent, '✈️Away')] = 'N/A'

# Display the results
results_df

Team          ARS               CHE               LIV               NOT  \
Location    🏠Home   ✈️Away    🏠Home   ✈️Away    🏠Home   ✈️Away    🏠Home   
WOL       📆 01/25    2 - 0    6 - 2  📆 01/20    2 - 1  📆 02/16  📆 01/06   
AVL         2 - 0  📆 01/18  📆 02/22    3 - 0  📆 03/15    2 - 0  📆 04/05   
EVE       📆 04/05    0 - 0    0 - 0  📆 04/26  📆 12/07  📆 04/02    2 - 0   
MCI         2 - 2  📆 02/02  📆 01/25    0 - 2  📆 02/23    2 - 0    0 - 3   
NOT       📆 02/25    3 - 0  📆 05/25    1 - 1  📆 01/14    0 - 1      N/A   
LIV       📆 05/10    2 - 2    1 - 2  📆 05/03      N/A      N/A    1 - 0   
NEW         0 - 1  📆 05/18  📆 05/10    2 - 1    3 - 3  📆 02/26  📆 02/23   
LEI       📆 02/15    4 - 2    2 - 1  📆 03/08  📆 04/19    3 - 1    3 - 1   
ARS           N/A      N/A  📆 03/15    1 - 1    2 - 2  📆 05/10    0 - 3   
BRE         3 - 1  📆 04/12  📆 04/05    2 - 1  📆 01/18    2 - 0    2 - 0   
WHU         5 - 2  📆 02/22    3 - 0  📆 02/03    5 - 0  📆 04/12  📆 05/18   
BHA       📆 01/04    1 - 1  📆 02/14    4 - 2  📆 05/18    2 - 1    2 - 2   
FUL         1 - 1  📆 04/01  📆 04/19    1 - 2  📆 04/05    2 - 2  📆 02/15   
CHE         1 - 1  📆 03/15      N/A      N/A  📆 05/03    2 - 1    1 - 1   
IPS       📆 04/19    1 - 0    0 - 2  📆 04/12    2 - 0  📆 01/25  📆 03/15   
CRY         5 - 1  📆 04/26  📆 01/04    1 - 1    1 - 0  📆 05/25  📆 05/03   
SOU       📆 05/25    3 - 1    5 - 1  📆 02/26    3 - 2  📆 03/08    1 - 0   
BOU         0 - 2  📆 05/03    1 - 0  📆 01/14  📆 02/01    3 - 0  📆 01/25   
TOT         1 - 0  📆 01/15    4 - 3  📆 04/02    6 - 3  📆 04/26  📆 04/19   
MUN       📆 03/08    2 - 0    1 - 1  📆 05/18    3 - 0  📆 01/05    3 - 2   

Team               
Location   ✈️Away  
WOL         1 - 1  
AVL         2 - 1  
EVE       📆 04/12  
MCI       📆 03/08  
NOT           N/A  
LIV       📆 01/14  
NEW         1 - 3  
LEI       📆 05/10  
ARS       📆 02/25  
BRE       📆 04/26  
WHU         3 - 0  
BHA       📆 02/01  
FUL         0 - 1  
CHE       📆 05/25  
IPS         1 - 0  
CRY         1 - 0  
SOU       📆 01/19  
BOU         1 - 1  
TOT         1 - 0  
MUN       📆 04/01

In [46]:
# Define a function to merge home and away columns
def merge_home_away(row):
    merged_row = {}
    for team in teams_of_interest:
        home_col = (team, '🏠Home')
        away_col = (team, '✈️Away')
        home_value = row[home_col]
        away_value = row[away_col]
        
        if not home_value.startswith('📆'):
            merged_row[team] = home_value
        elif not away_value.startswith('📆'):
            merged_row[team] = away_value
        else:
            merged_row[team] = ''
    
    return pd.Series(merged_row)

# Apply the function to each row in the DataFrame
merged_results = results_df.apply(merge_home_away, axis=1)

# Display the merged results
merged_results

,ARS,CHE,LIV,NOT
WOL,2 - 0,6 - 2,2 - 1,1 - 1
AVL,2 - 0,3 - 0,2 - 0,2 - 1
EVE,0 - 0,0 - 0,,2 - 0
MCI,2 - 2,0 - 2,2 - 0,0 - 3
NOT,3 - 0,1 - 1,0 - 1,N/A
LIV,2 - 2,1 - 2,N/A,1 - 0
NEW,0 - 1,2 - 1,3 - 3,1 - 3
LEI,4 - 2,2 - 1,3 - 1,3 - 1
ARS,N/A,1 - 1,2 - 2,0 - 3
BRE,3 - 1,2 - 1,2 - 0,2 - 0


In [47]:
# Define a function to color the results
def color_results(cell):
    if cell == 'N/A':
        return cell
    if cell.startswith('📆'):
        return f"<span style='color: gray; font-size: smaller;'>{cell}</span>"
    try:
        home_score, away_score = cell.split(' - ')
        home_score, away_score = int(home_score), int(away_score)
        if home_score > away_score:
            return f"<span style='color: green;'>🟢 {cell}</span>"
        elif home_score < away_score:
            return f"<span style='color: red;'>🔴 {cell}</span>"
        else:
            return f"<span style='color: gray;'>⚪ {cell}</span>"
    except:
        return cell

# Apply the function to each cell in the DataFrame
results_df_html = results_df.applymap(color_results)
mergedresults_df_html = merged_results.applymap(color_results)

# Append crest to each team name in the 'Location' column
results_df_html.index = results_df_html.index.map(lambda team: f" {team} <img src='{df_crests[df_crests['Team'] == team]['Crest'].values[0]}' width='16' height='16' style='vertical-align:middle;'/>")
mergedresults_df_html.index = mergedresults_df_html.index.map(lambda team: f" {team} <img src='{df_crests[df_crests['Team'] == team]['Crest'].values[0]}' width='16' height='16' style='vertical-align:middle;'/>")

# Update column names to include team crests
new_columns = []
for col in results_df_html.columns.levels[0]:
    if col in df_crests['Team'].values:
        crest_url = df_crests[df_crests['Team'] == col]['Crest'].values[0]
        for sub_col in results_df_html.columns.levels[1]:
            new_columns.append((f"<img src='{crest_url}' width='16' height='16' style='vertical-align:middle;'/> {col}", sub_col))
    else:
        for sub_col in results_df_html.columns.levels[1]:
            new_columns.append((col, sub_col))

results_df_html.columns = pd.MultiIndex.from_tuples(new_columns, names=results_df_html.columns.names)

# Update column names for mergedresults_df_html to include team crests
merged_new_columns = []
for col in mergedresults_df_html.columns:
    if col in df_crests['Team'].values:
        crest_url = df_crests[df_crests['Team'] == col]['Crest'].values[0]
        merged_new_columns.append(f"<img src='{crest_url}' width='16' height='16' style='vertical-align:middle;'/> {col}")
    else:
        merged_new_columns.append(col)

mergedresults_df_html.columns = merged_new_columns

# Sort the teams alphabetically
results_df_html = results_df_html.sort_index()
mergedresults_df_html = mergedresults_df_html.sort_index()

# Define a function to calculate points
def calculate_points(cell):
    if '🟢' in cell:
        return 3
    elif '⚪' in cell:
        return 1
    else:
        return 0

# Calculate points for each team
points_summary = results_df_html.applymap(calculate_points).sum()
mergedpoints_summary = mergedresults_df_html.applymap(calculate_points).sum()

# Create a summary row
summary_row = pd.DataFrame([points_summary], index=['Total Points'])
mergedsummary_row = pd.DataFrame([mergedpoints_summary], index=['Total Points'])

# Append the summary row to the DataFrame
results_df_html = pd.concat([results_df_html, summary_row])
mergedresults_df_html = pd.concat([mergedresults_df_html, mergedsummary_row])

# Update the index to include the summary row
results_df_html.index = results_df_html.index.map(lambda x: x if x != 'Total Points' else f"<b>{x}</b>")
mergedresults_df_html.index = mergedresults_df_html.index.map(lambda x: x if x != 'Total Points' else f"<b>{x}</b>")

# Render the DataFrame to HTML
html_output = results_df_html.to_html(escape=False)
HTML(html_output)

C:\Users\marian\AppData\Local\Temp\ipykernel_23060\3748643614.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  results_df_html = results_df.applymap(color_results)
C:\Users\marian\AppData\Local\Temp\ipykernel_23060\3748643614.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mergedresults_df_html = merged_results.applymap(color_results)
C:\Users\marian\AppData\Local\Temp\ipykernel_23060\3748643614.py:65: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  points_summary = results_df_html.applymap(calculate_points).sum()
C:\Users\marian\AppData\Local\Temp\ipykernel_23060\3748643614.py:66: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mergedpoints_summary = mergedresults_df_html.applymap(calculate_points).sum()


In [48]:
# Render the merged DataFrame to HTML
mergedhtml_output = mergedresults_df_html.to_html(escape=False)
HTML(mergedhtml_output)

,ARS,CHE,LIV,NOT
ARS,N/A,⚪ 1 - 1,⚪ 2 - 2,🔴 0 - 3
AVL,🟢 2 - 0,🟢 3 - 0,🟢 2 - 0,🟢 2 - 1
BHA,⚪ 1 - 1,🟢 4 - 2,🟢 2 - 1,⚪ 2 - 2
BOU,🔴 0 - 2,🟢 1 - 0,🟢 3 - 0,⚪ 1 - 1
BRE,🟢 3 - 1,🟢 2 - 1,🟢 2 - 0,🟢 2 - 0
CHE,⚪ 1 - 1,N/A,🟢 2 - 1,⚪ 1 - 1
CRY,🟢 5 - 1,⚪ 1 - 1,🟢 1 - 0,🟢 1 - 0
EVE,⚪ 0 - 0,⚪ 0 - 0,,🟢 2 - 0
FUL,⚪ 1 - 1,🔴 1 - 2,⚪ 2 - 2,🔴 0 - 1
IPS,🟢 1 - 0,🔴 0 - 2,🟢 2 - 0,🟢 1 - 0


In [51]:
# Transpose the merged results DataFrame
transposed_merged_results = mergedresults_df_html.T

# Function to split DataFrame into smaller DataFrames with a maximum of 5 columns each
def split_dataframe(df, max_columns=5):
    return [df.iloc[:, i:i + max_columns] for i in range(0, df.shape[1], max_columns)]

# Split the transposed merged results DataFrame
split_dfs = split_dataframe(transposed_merged_results, max_columns=5)

# Render each smaller DataFrame to HTML and concatenate the results
html_outputs = [df.to_html(escape=False) for df in split_dfs]
final_html_output = ''.join(html_outputs)

# Append the message to the final HTML output
final_html_output += "<span style='color: gray; font-size: smaller;'>Football data provided by the Football-Data.org API</span>"

# Display the concatenated HTML output
HTML(final_html_output)

,ARS,AVL,BHA,BOU,BRE
ARS,N/A,🟢 2 - 0,⚪ 1 - 1,🔴 0 - 2,🟢 3 - 1
CHE,⚪ 1 - 1,🟢 3 - 0,🟢 4 - 2,🟢 1 - 0,🟢 2 - 1
LIV,⚪ 2 - 2,🟢 2 - 0,🟢 2 - 1,🟢 3 - 0,🟢 2 - 0
NOT,🔴 0 - 3,🟢 2 - 1,⚪ 2 - 2,⚪ 1 - 1,🟢 2 - 0
,CHE,CRY,EVE,FUL,IPS
ARS,⚪ 1 - 1,🟢 5 - 1,⚪ 0 - 0,⚪ 1 - 1,🟢 1 - 0
CHE,N/A,⚪ 1 - 1,⚪ 0 - 0,🔴 1 - 2,🔴 0 - 2
LIV,🟢 2 - 1,🟢 1 - 0,,⚪ 2 - 2,🟢 2 - 0
NOT,⚪ 1 - 1,🟢 1 - 0,🟢 2 - 0,🔴 0 - 1,🟢 1 - 0
,LEI,LIV,MCI,MUN,NEW
